In [1]:
import pandas as pd

In [16]:
df = pd.read_csv("data/data_food_final.csv")
df = df.drop(columns = ['plus_code', 'business_status', 'icon','geometry', 'opening_hours','photos','vicinity', 'scope', 'reference'])
df.head()

,id,name,price_level,rating,types,user_ratings_total,lat,lng,closest_metro,closest_metro_km,Quantity of nearest museum,Quantity of nearest art_gallery,Quantity of nearest shopping_mall,Quantity of nearest places_tourist_attraction,Quantity of nearest same establishment,opportunity_take_away
0,ebe04f74ba360a7d600f0db89e9a4f1d926d428f,Feromon Efimova (Кальянная/Lounge бар/Тайм каф...,2.0,4.2,"['night_club', 'cafe', 'bar', 'restaurant', 'f...",212.0,59.924761,30.322215,['Спасская'],0.269347,35,10,34,53,94,0
1,8f3ea36295b8e3b349f04a9a06b0c73e9fd69f7d,Laundzh-Bar Kal'yan Khochu,NaN,4.8,"['night_club', 'cafe', 'bar', 'food', 'point_o...",139.0,59.925590,30.316788,['Садовая'],0.162308,44,11,31,58,116,0
2,fc621104fe8cf4fcf86a6a5e64dd8ee786b402b8,Sytaya Akula,NaN,3.9,"['cafe', 'bar', 'food', 'point_of_interest', '...",37.0,59.915517,30.309495,['Технологический институт-2'],0.518818,16,4,9,30,94,0
3,775029dc407f64cd8524c05ba8795d96382e4470,Кальянная Hustle,NaN,5.0,"['cafe', 'bar', 'restaurant', 'food', 'point_o...",27.0,59.924627,30.356393,['Лиговский проспект'],0.437038,36,35,34,52,119,0
4,f293aaa7b839c49817b6c12cb5dcfefb728f4398,СТОЯК БАР | всё по 200 р. | fix price bar,NaN,4.7,"['bar', 'cafe', 'restaurant', 'food', 'point_o...",47.0,59.925264,30.354044,['Владимирская'],0.423393,46,39,40,60,131,0


In [17]:
# считаем расстояние до центра
from geopy.distance import distance

def distance_to_center(index):
    try:
        return distance((df.loc[index].lat, df.loc[index].lng) ,(59.934199, 30.324456)).km
    except IndexError:
        return None

df['distance_to_center, km'] = list(map(lambda i: distance_to_center(i), df.index))

In [18]:
df = df[df['distance_to_center, km'] < 18]

In [19]:
df['result'] = df['rating'] * df['user_ratings_total']
# удаляем выбросы
df = df[df['result']< 20000]

In [20]:
# оставляем только данные, количество типов которых больше 50
df = df[df.groupby('types')['types'].transform('count').ge(50)]
df.head()

,id,name,price_level,rating,types,user_ratings_total,lat,lng,closest_metro,closest_metro_km,Quantity of nearest museum,Quantity of nearest art_gallery,Quantity of nearest shopping_mall,Quantity of nearest places_tourist_attraction,Quantity of nearest same establishment,opportunity_take_away,"distance_to_center, km",result
2,fc621104fe8cf4fcf86a6a5e64dd8ee786b402b8,Sytaya Akula,NaN,3.9,"['cafe', 'bar', 'food', 'point_of_interest', '...",37.0,59.915517,30.309495,['Технологический институт-2'],0.518818,16,4,9,30,94,0,2.243226,144.3
3,775029dc407f64cd8524c05ba8795d96382e4470,Кальянная Hustle,NaN,5.0,"['cafe', 'bar', 'restaurant', 'food', 'point_o...",27.0,59.924627,30.356393,['Лиговский проспект'],0.437038,36,35,34,52,119,0,2.080040,135.0
5,cc529e7d0038d813d0afd342936be45fff5bf822,Wood bar,2.0,4.7,"['cafe', 'bar', 'food', 'point_of_interest', '...",308.0,59.926036,30.352349,['Владимирская'],0.296679,52,43,49,67,136,0,1.805490,1447.6
9,aaf2e3a83371bdd09259f36f4128ee2c0770553a,Кальянная 1/2 OF YOU,NaN,4.7,"['cafe', 'bar', 'restaurant', 'food', 'point_o...",97.0,59.931000,30.359420,['Площадь Восстания'],0.075226,71,49,36,70,119,0,1.987175,455.9
11,45c6c021f483ca5850a1d97463ca43a8adae03ea,Антикафе-кальянная Terra 13 Lounge,NaN,4.7,"['cafe', 'bar', 'food', 'point_of_interest', '...",24.0,59.934828,30.335022,['Гостиный Двор'],0.111416,134,80,31,182,80,0,0.594867,112.8


In [21]:
# оставляем станции метро с количество кафе больше 20
df = df[df.groupby('closest_metro')['closest_metro'].transform('count').ge(20)]
df.head()

,id,name,price_level,rating,types,user_ratings_total,lat,lng,closest_metro,closest_metro_km,Quantity of nearest museum,Quantity of nearest art_gallery,Quantity of nearest shopping_mall,Quantity of nearest places_tourist_attraction,Quantity of nearest same establishment,opportunity_take_away,"distance_to_center, km",result
2,fc621104fe8cf4fcf86a6a5e64dd8ee786b402b8,Sytaya Akula,NaN,3.9,"['cafe', 'bar', 'food', 'point_of_interest', '...",37.0,59.915517,30.309495,['Технологический институт-2'],0.518818,16,4,9,30,94,0,2.243226,144.3
3,775029dc407f64cd8524c05ba8795d96382e4470,Кальянная Hustle,NaN,5.0,"['cafe', 'bar', 'restaurant', 'food', 'point_o...",27.0,59.924627,30.356393,['Лиговский проспект'],0.437038,36,35,34,52,119,0,2.080040,135.0
5,cc529e7d0038d813d0afd342936be45fff5bf822,Wood bar,2.0,4.7,"['cafe', 'bar', 'food', 'point_of_interest', '...",308.0,59.926036,30.352349,['Владимирская'],0.296679,52,43,49,67,136,0,1.805490,1447.6
9,aaf2e3a83371bdd09259f36f4128ee2c0770553a,Кальянная 1/2 OF YOU,NaN,4.7,"['cafe', 'bar', 'restaurant', 'food', 'point_o...",97.0,59.931000,30.359420,['Площадь Восстания'],0.075226,71,49,36,70,119,0,1.987175,455.9
11,45c6c021f483ca5850a1d97463ca43a8adae03ea,Антикафе-кальянная Terra 13 Lounge,NaN,4.7,"['cafe', 'bar', 'food', 'point_of_interest', '...",24.0,59.934828,30.335022,['Гостиный Двор'],0.111416,134,80,31,182,80,0,0.594867,112.8


In [22]:
# заменяем пересадочные станции на одну станцию
df["closest_metro"].replace({"['Спасская']": "['Садовая']", "['Сенная площадь']": "['Садовая']", "['Спортивная-2']":"['Спортивная-1']",
                             "['Технологический институт-2']" :"['Технологический институт-1']","['Владимирская']": "['Достоевская']",
                             "['Маяковская']": "['Площадь Восстания']","['Невский проспект']": "['Гостиный Двор']","['Звенигородская']": "['Пушкинская']",
                             "['Площадь Александра Невского-2']": "['Площадь Александра Невского-1']"}, inplace=True)
# смотрим количество по станциям метро (нужно учитывать расстояние между станциями)
df["closest_metro"].value_counts()

df["types"].replace({"['cafe', 'bar', 'food', 'point_of_interest', 'establishment']": "['cafe', 'bar', 'restaurant', 'food', 'point_of_interest', 'establishment']", \
                     "['bar', 'restaurant', 'food', 'point_of_interest', 'establishment']": "['bar', 'restaurant', 'food', 'point_of_interest', 'establishment']",\
                     "['cafe', 'food', 'point_of_interest', 'store', 'establishment']":"['cafe', 'food', 'point_of_interest', 'establishment']",\
                     "['meal_delivery', 'restaurant', 'food', 'point_of_interest', 'establishment']" :"['restaurant', 'meal_delivery', 'food', 'point_of_interest', 'establishment']",\
                     "['restaurant', 'meal_delivery', 'food', 'point_of_interest', 'establishment']":"['restaurant', 'food', 'point_of_interest', 'establishment']",\
                     "['restaurant', 'bar', 'food', 'point_of_interest', 'establishment']":"['bar', 'restaurant', 'food', 'point_of_interest', 'establishment']",\
                     "['bakery', 'cafe', 'food', 'point_of_interest', 'store', 'establishment']":"['cafe', 'bakery', 'food', 'point_of_interest', 'store', 'establishment']",\
                     "['meal_takeaway', 'restaurant', 'food', 'point_of_interest', 'establishment']":"['meal_delivery', 'meal_takeaway', 'restaurant', 'food', 'point_of_interest', 'establishment']",\
                    }, inplace=True)
# смотрим количество по станциям метро (нужно учитывать расстояние между станциями)
df["types"].value_counts()

['cafe', 'food', 'point_of_interest', 'establishment']                                            2301
['restaurant', 'food', 'point_of_interest', 'establishment']                                      2182
['bar', 'point_of_interest', 'establishment']                                                      566
['bar', 'restaurant', 'food', 'point_of_interest', 'establishment']                                306
['cafe', 'restaurant', 'food', 'point_of_interest', 'establishment']                               261
['cafe', 'bakery', 'food', 'point_of_interest', 'store', 'establishment']                          211
['meal_delivery', 'meal_takeaway', 'restaurant', 'food', 'point_of_interest', 'establishment']     153
['cafe', 'bar', 'restaurant', 'food', 'point_of_interest', 'establishment']                        124
['restaurant', 'meal_delivery', 'food', 'point_of_interest', 'establishment']                       70
['night_club', 'bar', 'point_of_interest', 'establishment']              

In [23]:
df["price_level"].value_counts()

2.0    2286
1.0     342
3.0      93
4.0       8
Name: price_level, dtype: int64

In [24]:
df.isna().sum()

id                                                  0
name                                                0
price_level                                      3495
rating                                              0
types                                               0
user_ratings_total                                  0
lat                                                 0
lng                                                 0
closest_metro                                       0
closest_metro_km                                    0
Quantity of nearest museum                          0
Quantity of nearest art_gallery                     0
Quantity of nearest shopping_mall                   0
Quantity of nearest places_tourist_attraction       0
Quantity of nearest same establishment              0
opportunity_take_away                               0
distance_to_center, km                              0
result                                              0
dtype: int64

In [25]:
df.shape

(6224, 18)

In [26]:
# убрать лишнее из названий типов
# df.types = df.types.apply(lambda x: ', '.join(x[1:-1].split(',')[:-2]))

In [27]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 6224 entries, 2 to 8086
Data columns (total 18 columns):
 #   Column                                         Non-Null Count  Dtype  
---  ------                                         --------------  -----  
 0   id                                             6224 non-null   object 
 1   name                                           6224 non-null   object 
 2   price_level                                    2729 non-null   float64
 3   rating                                         6224 non-null   float64
 4   types                                          6224 non-null   object 
 5   user_ratings_total                             6224 non-null   float64
 6   lat                                            6224 non-null   float64
 7   lng                                            6224 non-null   float64
 8   closest_metro                                  6224 non-null   object 
 9   closest_metro_km                               6224 

In [31]:
df.rename(columns=
          {"Quantity of nearest museum":                    "n_nearest_museums", 
           "Quantity of nearest art_gallery":               "n_nearest_art_galleries",
           "Quantity of nearest shopping_mall":             "n_nearest_shopping_malls",
           "Quantity of nearest places_tourist_attraction": "n_nearest_tourist_attractions",
           "Quantity of nearest same establishment":        "n_nearest_same_establishments",
           "distance_to_center, km":                        "distance_to_center_km"}, inplace=True)

In [33]:
df.to_csv('data/data_cafe_prep.csv', index=False)

In [32]:
df

,id,name,price_level,rating,types,user_ratings_total,lat,lng,closest_metro,closest_metro_km,n_nearest_museums,n_nearest_art_galleries,n_nearest_shopping_malls,n_nearest_tourist_attractions,n_nearest_same_establishments,opportunity_take_away,distance_to_center_km,result
2,fc621104fe8cf4fcf86a6a5e64dd8ee786b402b8,Sytaya Akula,NaN,3.9,"['cafe', 'bar', 'restaurant', 'food', 'point_o...",37.0,59.915517,30.309495,['Технологический институт-1'],0.518818,16,4,9,30,94,0,2.243226,144.3
3,775029dc407f64cd8524c05ba8795d96382e4470,Кальянная Hustle,NaN,5.0,"['cafe', 'bar', 'restaurant', 'food', 'point_o...",27.0,59.924627,30.356393,['Лиговский проспект'],0.437038,36,35,34,52,119,0,2.080040,135.0
5,cc529e7d0038d813d0afd342936be45fff5bf822,Wood bar,2.0,4.7,"['cafe', 'bar', 'restaurant', 'food', 'point_o...",308.0,59.926036,30.352349,['Достоевская'],0.296679,52,43,49,67,136,0,1.805490,1447.6
9,aaf2e3a83371bdd09259f36f4128ee2c0770553a,Кальянная 1/2 OF YOU,NaN,4.7,"['cafe', 'bar', 'restaurant', 'food', 'point_o...",97.0,59.931000,30.359420,['Площадь Восстания'],0.075226,71,49,36,70,119,0,1.987175,455.9
11,45c6c021f483ca5850a1d97463ca43a8adae03ea,Антикафе-кальянная Terra 13 Lounge,NaN,4.7,"['cafe', 'bar', 'restaurant', 'food', 'point_o...",24.0,59.934828,30.335022,['Гостиный Двор'],0.111416,134,80,31,182,80,0,0.594867,112.8
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8082,e2800fa2050e310bab69b2cfbd1d1a43510db6b6,Brewgge,2.0,4.8,"['bar', 'point_of_interest', 'establishment']",62.0,60.070492,30.327381,['Парнас'],0.528913,0,0,5,0,8,0,15.185564,297.6
8083,78fd775f013a24dd3231ec43b2322cae97400d97,Woodpecker bar,NaN,4.9,"['bar', 'point_of_interest', 'establishment']",25.0,60.069987,30.326108,['Парнас'],0.547118,0,0,5,0,8,0,15.128704,122.5
8084,f7ecaaaa80eac7f7e213887bac5cddd94797658f,Клуб виртуальной реальности | HI-TECH LOUNGE U...,NaN,4.5,"['night_club', 'bar', 'point_of_interest', 'es...",8.0,60.076130,30.342669,['Парнас'],1.107719,2,2,5,0,8,0,15.845439,36.0
8085,6b5e4aea00ca4ed40ebcd1a3aba606df1e23a14f,SV Groupe,NaN,4.3,"['night_club', 'bar', 'point_of_interest', 'es...",11.0,60.072393,30.361603,['Парнас'],1.637602,4,0,0,0,2,0,15.535391,47.3


In [34]:
df.to_json('data/data_cafe_prep.json', orient='records', force_ascii=False, lines=True)